<font size=6, color=blue>NLP обработка данных о сотрудниках ВУЗа</font>

# Цель работы и исходные данные

## Задача:

Из текста в CSV файле необходимо распарсить имя человека и его должность.

В колонке json храниться 2 ключа “left” и “right” , при конкатенации этих  значений получается текст, в котором теоретически может храниться информация о сотруднике ВУЗ.
	Например из вот такой строки:
“
Department Contact:
Jodi Musser, Program Director
509-963-2773
“
Необходимо вытащить информацию в таком виде:
⦁	first_name: Jodi
⦁	last_name: Musser
⦁	job_title: Program Director

## Исходные данные

⦁	CSV файл с данными data.csv

⦁	список ключевиков с названиями потенциальных должностей https://drive.google.com/file/d/1vs3MQr-DvklQ9tmTQP9zYdDfV6fzPzBI/view?usp=sharing

## Требования к выполнению

Требования к выполнению задания:

⦁	При разработке использовать библиотеку SpaCy

⦁	Скрипт должен работать максимально оптимизировано. Каждый новый запрос в модель должен выполняться < 1c.

⦁	При поиске должностей следует использовать spacy.Matcher (https://spacy.io/usage/rule-based-matching) для выявления нечетких совпадений между заданными ключевиками и должностями в заданном тексте.

⦁	Результатом работы должен быть тот же CSV файл, но с заполненными полями:
⦁	first_name
⦁	last_name
⦁	job_title

# Исследовательский анализ и чистка данных

## Импорт необходимых библиотек

In [2]:
#Import libraries
import pandas as pd
import spacy

#Import Matcher class
from spacy.matcher import Matcher

#Load model for english language
nlp = spacy.load("en_core_web_sm")

## Загрузка набора данных 

In [3]:
data=pd.read_csv('data.csv')

## EDA

In [4]:
#Take a look at the data
data.head()

,id,url,email,json,title,first_name,last_name,academic_title,department,school,processed,created_at,updated_at
0,1,https://www.abac.edu/,vfenn@abac.edu,"{""left"": "" the winner. The number on each ball...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24
1,2,https://www.abac.edu/,bray@abac.edu,"{""left"": ""er person and can be purchased onlin...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24
2,3,https://www.abac.edu/,admissions@abac.edu,"{""left"": ""ty, Prince Automotive Group, Rotary ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24
3,4,https://www.abac.edu/,webmaster@abac.edu,"{""left"": ""mics\nRegistrar\nTranscript Request\...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24
4,5,https://www.alu.edu/,admissions@alu.edu,"{""left"": ""Abraham Lincoln University & Online ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24


In [5]:
data.loc[0,'json']

'{"left": " the winner. The number on each ball will be associated with an\\nindividual that purchased chances to win. The grand prize is a check with a\\ndesignated value equal to 50 per cent (nearest $10) of the numbered golf balls\\nsold.\\nTo\\nparticipate in the tournament or the ball drop event, interested persons can\\ncontact Fenn at (229) 391-5067, email her at ", "right": "\\n, or register online at https://www.abac.edu/academics/sanr-classic/ .\\n###\\nBaldwin Players Announce Cast for ABAC Fall Production\\nSeptember 3 2019\\nBaldwin Players Announce Cast for ABAC Fall Production\\nTIFTONâ€”Baldwin\\nPlayersâ€™ Director Brian Ray has announced the cast for the theatre troupeâ€™s\\nupcoming production of â€œBoeing, Boeingâ€\x9d by Mark Camolett"}'

In [6]:
data.shape

(99924, 13)

Датасет содержит 99924 записи, 13 столбцов. Нас интересует столбец json. 

Проверим наличие пустых значений.

In [7]:
data.json.isna().any()

False

Столбец не содержит пустых значений. Проверим по нему дубликаты. 

In [8]:
def duplicates(df):
    dupl=df.duplicated(subset=['json'])
    s=0
    for ind in dupl.index:
        if dupl.loc[ind]==True:
            s+=1
    print('{} duplicates'.format(s))
duplicates(data)

42573 duplicates


Поскольку задача состоит в извлечении информации только из столбца 'json' без связи с данными других столбцов, то дубликаты по нему можно удалить.


## Чистка данных

In [9]:
#Drop duplicates
data=data.drop_duplicates(subset=['json'], ignore_index=False)

In [10]:
data.shape

(57351, 13)

После удаления дубликатов датасет содержит 57351 строку.

Переименуем столбец 'title' в соответствии с заданием в 'job_title'.

In [11]:
data.rename(columns = {'title':'job_title'}, inplace = True) 

Выполним конкатенацию данных в столбце 'json'.

In [12]:
import json
for idx in data.index:
    data_json = json.loads(data['json'][idx])
    json_concat=data_json['left']+data_json['right']
    data['json'][idx]=json_concat

<ipython-input-12-991f7defa32e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['json'][idx]=json_concat


In [13]:
data.head()

,id,url,email,json,job_title,first_name,last_name,academic_title,department,school,processed,created_at,updated_at
0,1,https://www.abac.edu/,vfenn@abac.edu,the winner. The number on each ball will be a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24
1,2,https://www.abac.edu/,bray@abac.edu,er person and can be purchased online at www.p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24
2,3,https://www.abac.edu/,admissions@abac.edu,"ty, Prince Automotive Group, Rotary Club of\nT...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24
3,4,https://www.abac.edu/,webmaster@abac.edu,mics\nRegistrar\nTranscript Request\nAcademic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24
4,5,https://www.alu.edu/,admissions@alu.edu,Abraham Lincoln University & Online Law School...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24


In [14]:
data=data.reset_index(drop=True)

# Извлечение данных из текста с помощью SpaCy

## Алгоритм работы с данными

1. Определить список должностей, которые потенциально могут содержаться в тексте (он один для всех записей, задан в исходных данных)
2. Определить перечень имён, которые встречаются в данном тексте (он уникален для каждой записи, его мы будем формировать с помощью распознавания имён - Named Entity Recognition)
3. Составить паттерны "должность+имя" для данного текста
4. С помощью Matcher найти эти паттерны в тексте
5. Распарсить результат совпадений и поместить в заданные столбцы

## Подготовка токенов

Используя предоставленную в задании функцию, получим список потенциальных должностей.

In [15]:
from keywords import keywords
tokens=keywords()
tokens=tokens['academic_title']

In [17]:
tokens

['Distinguished Professor',
 'Professor',
 'Associate Professor',
 'Assistant Professor',
 'Adjunct Professor',
 'Senior Lecturer',
 'Lecturer',
 'Associate Lecturer',
 'Assistant Lecturer',
 'Professor Emeritus',
 'Emeritus Professor',
 'Dean',
 'Head',
 'Chair',
 'Director',
 'Provost',
 'Instructor',
 'Coach',
 'Coordinator',
 'Manager',
 'Counselor',
 'Trainer',
 'Cashier',
 'President',
 'Researcher',
 'Assistant',
 'Research Associate',
 'Counselor',
 'Adviser',
 'VP',
 'Analyst',
 'Officer',
 'Chief']

В списке есть должности как состоящие из одного слова, так и из двух. Поэтому при составлении паттернов нужно будет учесть оба варианта. Для этого из списка tokens сделаем 3 разных списка: , первый с первым словом из двухсложных названий, второй - со вторым словом из них, третий с однословными должностями. 

In [18]:
first_titles=[title.split(' ')[0] for title in tokens if ' ' in title]
last_titles=[title.split(' ')[1] for title in tokens if ' ' in title]
only_first_titles=[title for title in tokens if ' ' not in title]

print('first_titles:', first_titles) 
print('last_titles: ',last_titles)
print('only_first_titles:',only_first_titles)

first_titles: ['Distinguished', 'Associate', 'Assistant', 'Adjunct', 'Senior', 'Associate', 'Assistant', 'Professor', 'Emeritus', 'Research']
last_titles:  ['Professor', 'Professor', 'Professor', 'Professor', 'Lecturer', 'Lecturer', 'Lecturer', 'Emeritus', 'Professor', 'Associate']
only_first_titles: ['Professor', 'Lecturer', 'Dean', 'Head', 'Chair', 'Director', 'Provost', 'Instructor', 'Coach', 'Coordinator', 'Manager', 'Counselor', 'Trainer', 'Cashier', 'President', 'Researcher', 'Assistant', 'Counselor', 'Adviser', 'VP', 'Analyst', 'Officer', 'Chief']


## Извлечение паттернов из одной записи

При написании скрипта следует учесть возможные варианты:
1. Запись не содержит ни должностей, ни имён.
2. Запись не содержит должностей, но содержит имена.
3. Запись содержит должность(и), но без имени
4. Запись содержит должность+имя
5. Запись содержит несколько паттернов "должность+имя"

По смыслу задания будем брать в рассмотрение только варианты 4 и 5. Для случаев 1-3 мы будем заполнять поля значениями 'Not found'. 

Кроме того, следует учесть, что имена также могут состоять только из одного слова или двух (трёхсложный вариант пока не рассматриваем).

В случае варианта 5 примем решение заносить последовательность извлечённых должностей (список) в ячейку соотвествующей строки столбца 'job_title'. Аналогично поступим с именами, отвечающими этим должностям. Таким образом в результате выполнения скрипта мы заполняем 3 листа (job_title, first_name, last_name), элементами которых тоже будут листы. 

Также для начала согласимся, что если найденная моделью сущность "PERSON" состоит из одного слова, то это будет  first_name.  В данном случае в last_name мы заносим 'Not found'. После обработки датасета можно проанализировать полученные данные и поменять этот момент.

В первом приближении будем искать только паттерны вида "должность имя" (как в задании). При необходимости можно будет создать более сложные, например: "{токен из должностей} of {токен из областей науки} имя". 
В скрипт также добавлены команды для замера времени выполнения запроса.


In [21]:
# Lists to store the extracted data
l=len(data)
job_title=[[] for _ in range(l)]
data_first_names=[[] for _ in range(l)]
data_last_names=[[] for _ in range(l)]
spans=[]

#Script timing
import time
start_time = time.time()

#Test the script on the first row of dataset
index_of_text_to_test_on = 0
text_to_test_on = data.json.iloc[index_of_text_to_test_on]
idx=index_of_text_to_test_on 

#Process text with nlp-model
doc = nlp(text_to_test_on)

#Recognize named entities in doc and create lists of names
persons=[entity.text for entity in doc.ents if entity.label_=="PERSON"]
            
first_names=[person.split(' ')[0] for person in persons if ' ' in person]
last_names=[person.split(' ')[1] for person in persons if ' ' in person]
only_first_names=[person for person in persons if ' ' not in person]

print(first_names) 
print(last_names)
print(only_first_names)  

#Create Matcher object        
matcher = Matcher(nlp.vocab)

#Create the patterns:
#1-word title+first_name+last_name
pattern1 = [{"TEXT": {"IN": only_first_titles}},{"TEXT":{"IN": first_names}},{"TEXT":{"IN":last_names}}]
#1-word title+1-word name (first_name)
pattern2=[{"TEXT": {"IN": only_first_titles}},{"TEXT":{"IN": only_first_names}}]
#2-words title+first_name+last_name
pattern3 = [{"TEXT": {"IN": first_titles}},{"TEXT": {"IN": last_titles}},{"TEXT":{"IN": first_names}},{"TEXT":{"IN":last_names}}]
#2-words title+1-word name (first_name)
pattern4=[{"TEXT": {"IN": first_titles}},{"TEXT": {"IN": last_titles}},{"TEXT":{"IN": only_first_names}}]

#Add pattern rules to the matcher
matcher.add("JSON", None, pattern1,pattern2,pattern3,pattern4)

#Find matches
matches = matcher(doc)

#Parse results to corrresponding lists
if not matches:
                job_title[idx]='Not found'
                data_first_names[idx]='Not found'
                data_last_names[idx]='Not found'
else:
                spans = [doc[start:end] for _, start, end in matches]
                for span in spacy.util.filter_spans(spans): 
                    print(span.start, span.end, span.text)

                    if span.end-span.start==2:  #1-word job-title, 1-word name (only first name)
                        job_title[idx].append(doc[span.start])
                        data_first_names[idx].append(doc[span.start+1]) 
                        data_last_names[idx].append('Not found')  
                    elif span.end-span.start==4:  #2-words job-title, 2-words name (first name, last name)
                        job_title[idx].append(str(doc[span.start])+' '+str(doc[span.start+1]))
                        data_first_names[idx].append(doc[span.start+2]) 
                        data_last_names[idx].append(doc[span.start+3])
                    else:  #Check if 2 first words are name of job_title             
                        if str(doc[span.start:span.start+2]) in tokens:
                            #2-word job-title, 1-words name (first name) 
                            job_title[idx].append(str(doc[span.start])+' '+str(doc[span.start+1]))
                            data_first_names[idx].append(doc[span.start+2])  
                            data_last_names[idx].append('Not found')  
                        else:#1-word job-title, 2-words name (first name, last name)       
                            job_title[idx].append(doc[span.start]) 
                            data_first_names[idx].append(doc[span.start+1]) 
                            data_last_names[idx].append(doc[span.start+2])    
                                                  
print("--- %s seconds ---" % (time.time() - start_time))   

['Brian', 'Mark']
['Ray', 'Camolett']
['Fenn']
120 123 Director Brian Ray
--- 0.03298139572143555 seconds ---


Время выполнения 1 запроса (0.034 с) не превышает требуемого. 

## Извлечение данных из всего набора

In [25]:
l=len(data)
job_title=[[] for _ in range(l)]
data_first_names=[[] for _ in range(l)]
data_last_names=[[] for _ in range(l)]
spans=[]


for idx,string in data.iterrows():
        doc = nlp(string.json)

        persons=[entity.text for entity in doc.ents if entity.label_=="PERSON"]   
        first_names=[person.split(' ')[0] for person in persons if ' ' in person]
        last_names=[person.split(' ')[1] for person in persons if ' ' in person]
        only_first_names=[person for person in persons if ' ' not in person]

        matcher = Matcher(nlp.vocab)

        pattern1 = [{"TEXT": {"IN": only_first_titles}},{"TEXT":{"IN": first_names}},{"TEXT":{"IN":last_names}}]
        pattern2=[{"TEXT": {"IN": only_first_titles}},{"TEXT":{"IN": only_first_names}}]
        pattern3 = [{"TEXT": {"IN": first_titles}},{"TEXT": {"IN": last_titles}},{"TEXT":{"IN": first_names}},{"TEXT":{"IN":last_names}}]
        pattern4=[{"TEXT": {"IN": first_titles}},{"TEXT": {"IN": last_titles}},{"TEXT":{"IN": only_first_names}}]

        matcher.add("JSON", None, pattern1,pattern2,pattern3,pattern4)

        matches = matcher(doc)

        if not matches:
                    job_title[idx]='Not found'
                    data_first_names[idx]='Not found'
                    data_last_names[idx]='Not found'
        else:
                    spans = [doc[start:end] for _, start, end in matches]
                    for span in spacy.util.filter_spans(spans): 

                        if span.end-span.start==2:  #1-word job-title, 1-word name (only first name)
                            job_title[idx].append(doc[span.start])
                            data_first_names[idx].append(doc[span.start+1]) 
                            data_last_names[idx].append('Not found')  
                        elif span.end-span.start==4:  #2-words job-title, 2-words name (first name, last name)
                            job_title[idx].append(str(doc[span.start])+' '+str(doc[span.start+1]))
                            data_first_names[idx].append(doc[span.start+2]) 
                            data_last_names[idx].append(doc[span.start+3])
                        else:  #Check if 2 first words are name of job_title             
                            if str(doc[span.start:span.start+2]) in tokens:  #2-word job-title, 1-words name (first name) 
                                job_title[idx].append(str(doc[span.start])+' '+str(doc[span.start+1]))
                                data_first_names[idx].append(doc[span.start+2])  
                                data_last_names[idx].append('Not found')  
                            else:#1-word job-title, 2-words name (first name, last name)       
                                job_title[idx].append(doc[span.start]) 
                                data_first_names[idx].append(doc[span.start+1]) 
                                data_last_names[idx].append(doc[span.start+2])     



In [26]:
#Check that lists have correct length                                
print('job_title:',len(job_title),'data_first_names:',len(data_first_names),'data_last_names:',len(data_last_names))  

job_title: 57351 data_first_names: 57351 data_last_names: 57351


Распарсим полученные результаты в датасет. Для удобства скопируем исходный в новый data_filled, в который и поместим результаты.

In [28]:
data_filled=data.copy()

In [29]:
#Transform lists it into strings
for idx in data_filled.index:
    if job_title[idx]=='Not found':
        data_filled['job_title'][idx]='Not found'
    else: data_filled['job_title'][idx]=', '.join([str(elem) for elem in job_title[idx]]) 
    if data_first_names[idx]=='Not found':
        data_filled['first_name'][idx]='Not found'
    else: data_filled['first_name'][idx]=', '.join([str(elem) for elem in data_first_names[idx]]) 
    if data_last_names[idx]=='Not found':
        data_filled['last_name'][idx]='Not found'
    else: data_filled['last_name'][idx]=', '.join([str(elem) for elem in data_last_names[idx]])     

<ipython-input-29-21885e009796>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: data_filled['job_title'][idx]=', '.join([str(elem) for elem in job_title[idx]])
C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-29-21885e009796>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: data_filled['first_name']

In [30]:
data_filled.head()

,id,url,email,json,job_title,first_name,last_name,academic_title,department,school,processed,created_at,updated_at
0,1,https://www.abac.edu/,vfenn@abac.edu,the winner. The number on each ball will be a...,Director,Brian,Ray,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24
1,2,https://www.abac.edu/,bray@abac.edu,er person and can be purchased online at www.p...,Not found,Not found,Not found,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24
2,3,https://www.abac.edu/,admissions@abac.edu,"ty, Prince Automotive Group, Rotary Club of\nT...",Not found,Not found,Not found,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24
3,4,https://www.abac.edu/,webmaster@abac.edu,mics\nRegistrar\nTranscript Request\nAcademic ...,Not found,Not found,Not found,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24
4,5,https://www.alu.edu/,admissions@alu.edu,Abraham Lincoln University & Online Law School...,Not found,Not found,Not found,NaN,NaN,NaN,NaN,2019-09-16 11:37:24,2020-02-06 03:33:24


Сохраняем таблицу с результатами в csv файл.

In [44]:
data_filled.to_csv('data_filled.csv')

# Анализ результатов

Определим количество записей, где не найдены должности.

In [32]:
t=0
for idx in data_filled.index:
    if data_filled['job_title'][idx]=='Not found':
        t=t+1
        t_perc=100*t/l
print ('\'Not found\' in job_title {0} - {1:2.1f}%'.format(t,t_perc))

'Not found' in job_title 54656 - 95.3%


In [33]:
data_filled['job_title'].describe()

count         57351
unique          104
top       Not found
freq          54656
Name: job_title, dtype: object

Найдено 104 разных должности. Однако большой процент пропусков наводит на мысль о существовании в тексте паттернов, которые мы не учли. 
Например, можно проверить, есть ли в тексте сочетания "Professor of".  

In [38]:
s=len([data['json'][idx] for idx in data.index if 'Professor of' in data['json'][idx]])
print(s)        

5889


Найдено 5889 записей с заданной комбинацией слов, которые могут быть полезной информацией. Таким образом, необходимо дополнять модель более широкими вариациями паттернов. 

Оценим качество распознавания имён:

In [39]:
print([data_filled['first_name'][idx] for idx in data_filled.index if data_filled['first_name'][idx]!='Not found'])

['Brian', 'Emeritus', 'James', 'James', 'Jalen', 'Todd, Manfred', 'Marion', 'Timothy', 'David', 'Karen', 'Upward', 'Upward', 'Upward', 'Upward', 'Ross', 'Dean', 'Dean, Registrar', 'Dean, Registrar', 'Dean, Registrar', 'Quinn, Quinn', 'Direc', 'Registrar', 'Registrar', 'Registrar', 'Registrar', 'Registrar', 'Registrar', 'Registrar', 'Randy', 'Emeritus, Emeritus, Emeritus, Emeritus, Emeritus, Emeritus', 'Emeritus, Emeritus, Emeritus, Emeritus, Emeritus, Emeritus', 'Emeritus, Emeritus, Emeritus', 'Emeritus, Emeritus, Lat', 'Athletic', 'Athletic', 'Athletic', 'Athletic', 'Athletic', 'CIS', 'CIS', 'CIS', 'CIS', 'CIS', 'CIS', 'Basebal', 'Registrar', 'Registrar', 'Registrar', 'Registrar', 'Athletic', 'Athletic', 'Athletic, Men', 'Athletic, Men', 'Athletic, Men', 'Men', 'Men', 'Men', 'Athletic', 'Athletic', 'Athletic', 'Athletic', 'Emeritus', 'Emeritus, Dean', 'Emeritus', 'Emeritus', 'Emeritus, Dean, Dean', 'Emeritus, Dean, Dean', 'Dean, Dean', 'Dean', 'Dean', 'Dean', 'Link', 'Hilary', 'Link',

Как видно, NER индентифицирует в качестве имён много других сущностей. Поэтому модель нуждается в дообучении: https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/


# Итоги

После удаления дубликатов, модель извлекла должности и имена сотрудников примерно из 5% записей.  

Как показал анализ, записи содержат также другие варианты интересующих нас паттернов (фразы с другими комбинациями имён и должностей).

Поскольку в скрипте использовалась дефолтная NER модель, качество выявления сущностей невысокое. 

Профессии могут распознаваться как часть имён и наоборот. 

ВОЗМОЖНОСТИ УСОВЕРШЕНСТВОВАНИЯ АЛГОРИТМА:

1). Разработать базу из нескольких сотен примеров с именами и должностями, на которых обучить NER

2). Написать ещё несколько вариантов паттернов для выявления должностей, например: "Professor of History John Gray", "John Gray the Professor". Для этого предварительно составить массив с потенциальными областями науки (история, математика и т.д). Можно разработать паттерны для трёхсложных имён.

3). Если важна привязка к датам получения информации, можно не удалять из датасета дубликаты по столбцу json.


# P.S.

Разработаннный код в функциях значительно увеличивает время обработки датасета.

In [41]:
# Lists to store the extracted data
l=len(data)
print(l)
job_title=[[] for _ in range(l)]
data_first_names=[[] for _ in range(l)]
data_last_names=[[] for _ in range(l)]
spans=[]

#Script timing
import time
start_time = time.time()

#Test the script on the first row of dataset
index_of_review_to_test_on = 0
text_to_test_on = data.json.iloc[index_of_review_to_test_on]
idx=index_of_review_to_test_on

#Process text with nlp-model
doc = nlp(text_to_test_on)

#Recognize named entities in doc and create lists of names
def get_names(doc):
    persons=[entity.text for entity in doc.ents if entity.label_=="PERSON"]
            
    first_names=[person.split(' ')[0] for person in persons if ' ' in person]
    last_names=[person.split(' ')[1] for person in persons if ' ' in person]
    only_first_names=[person for person in persons if ' ' not in person]

    return first_names,last_names,only_first_names

first_names,last_names,only_first_names=get_names(doc)

print(first_names) 
print(last_names)
print(only_first_names)  

#Create Matcher object        
matcher = Matcher(nlp.vocab)

#Create the patterns:
def create_patterns(first_titles,last_titles,only_first_titles,first_names,last_names,only_first_names):
     #1-word title+first_name+last_name
    pattern1 = [{"TEXT": {"IN": only_first_titles}},{"TEXT":{"IN": first_names}},{"TEXT":{"IN":last_names}}]
     #1-word title+1-word name (first_name)
    pattern2=[{"TEXT": {"IN": only_first_titles}},{"TEXT":{"IN": only_first_names}}]
     #2-words title+first_name+last_name
    pattern3 = [{"TEXT": {"IN": first_titles}},{"TEXT": {"IN": last_titles}},{"TEXT":{"IN": first_names}},{"TEXT":{"IN":last_names}}]
     #2-words title+1-word name (first_name)
    pattern4=[{"TEXT": {"IN": first_titles}},{"TEXT": {"IN": last_titles}},{"TEXT":{"IN": only_first_names}}]

    return pattern1,pattern2,pattern3,pattern4

pattern1,pattern2,pattern3,pattern4=create_patterns(first_titles,last_titles,only_first_titles,first_names,last_names,only_first_names)

#Add pattern rules to the matcher
matcher.add("JSON", None, pattern1,pattern2,pattern3,pattern4)

#Find matches
matches = matcher(doc)

#Parse results to corrresponding lists
def parse_matches(matches):
    if not matches:
                job_title[idx]='Not found'
                data_first_names[idx]='Not found'
                data_last_names[idx]='Not found'
    else:
                spans = [doc[start:end] for _, start, end in matches]
                for span in spacy.util.filter_spans(spans): 
                    print(span.start, span.end, span.text)

                    if span.end-span.start==2:  #1-word job-title, 1-word name (only first name)
                        job_title[idx].append(doc[span.start])
                        data_first_names[idx].append(doc[span.start+1]) 
                        data_last_names[idx].append('Not found')  
                    elif span.end-span.start==4:  #2-words job-title, 2-words name (first name, last name)
                        job_title[idx].append(str(doc[span.start])+' '+str(doc[span.start+1]))
                        data_first_names[idx].append(doc[span.start+2]) 
                        data_last_names[idx].append(doc[span.start+3])
                    else:  #Check if 2 first words are name of job_title             
                        if str(doc[span.start:span.start+2]) in tokens:
                            #2-word job-title, 1-words name (first name) 
                            job_title[idx].append(str(doc[span.start])+' '+str(doc[span.start+1]))
                            data_first_names[idx].append(doc[span.start+2])  
                            data_last_names[idx].append('Not found')  
                        else:#1-word job-title, 2-words name (first name, last name)       
                            job_title[idx].append(doc[span.start]) 
                            data_first_names[idx].append(doc[span.start+1]) 
                            data_last_names[idx].append(doc[span.start+2])
                            
    return job_title, data_first_names,data_last_names                        

job_title[idx], data_first_names[idx],data_last_names[idx]=parse_matches(matches)

print("--- %s seconds ---" % (time.time() - start_time))   

57351
['Brian', 'Mark']
['Ray', 'Camolett']
['Fenn']
120 123 Director Brian Ray
--- 0.04097723960876465 seconds ---


In [42]:
#Change parse_matches function for not printing the span.text
def parse_matches(matches,idx):
    idx=idx
    if not matches:
                job_title[idx]='Not found'
                data_first_names[idx]='Not found'
                data_last_names[idx]='Not found'
    else:
                spans = [doc[start:end] for _, start, end in matches]
                for span in spacy.util.filter_spans(spans): 
                                            
                    if span.end-span.start==2:  #1-word job-title, 1-word name (only first name)
                        job_title[idx].append(doc[span.start])
                        data_first_names[idx].append(doc[span.start+1]) 
                        data_last_names[idx].append('Not found')  
                    elif span.end-span.start==4:  #2-words job-title, 2-words name (first name, last name)
                        job_title[idx].append(str(doc[span.start])+' '+str(doc[span.start+1]))
                        data_first_names[idx].append(doc[span.start+2]) 
                        data_last_names[idx].append(doc[span.start+3])
                    else:  #Check if 2 first words are name of job_title             
                        if str(doc[span.start:span.start+2]) in tokens:
                            #2-word job-title, 1-words name (first name) 
                            job_title[idx].append(str(doc[span.start])+' '+str(doc[span.start+1]))
                            data_first_names[idx].append(doc[span.start+2])  
                            data_last_names[idx].append('Not found')  
                        else:#1-word job-title, 2-words name (first name, last name)       
                            job_title[idx].append(doc[span.start]) 
                            data_first_names[idx].append(doc[span.start+1]) 
                            data_last_names[idx].append(doc[span.start+2])
                            
    return job_title[idx], data_first_names[idx],data_last_names[idx]  

In [ ]:
l=len(data)
job_title=[[] for _ in range(l)]
data_first_names=[[] for _ in range(l)]
data_last_names=[[] for _ in range(l)]
spans=[]

for idx,string in data.iterrows():

    doc = nlp(string.json)
    first_names,last_names,only_first_names=get_names(doc)
    pattern1,pattern2,pattern3,pattern4=create_patterns(first_titles,last_titles,only_first_titles,first_names,last_names,only_first_names)
    matcher.add("JSON", None, pattern1,pattern2,pattern3,pattern4)
    matches = matcher(doc)
    job_title[idx], data_first_names[idx],data_last_names[idx]=parse_matches(matches,idx) 